# Import Packages

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing import image

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

import warnings
warnings.filterwarnings('ignore')

# Load the pretrained network like VGG16,ALexNet,VGG19 etc

In [2]:
model_mobilenet_conv = MobileNet(weights='imagenet', include_top=False)

17227776/17225924 [==============================] - 10s 1us/step


# Freeze the model, No weigh update, Use Trained Weights

In [65]:
for layer in model_mobilenet_conv.layers:
    layer.trainable = False

# Training Parameters

In [66]:
img_width, img_height = 224, 224
train_data_dir = 'Train'
val_data_dir = 'Validate'
model_weights_file = 'mobilenet.h5'
nb_train_samples = 20
nb_val_samples = 4
nb_epochs = 150

# Build a classification model on top of Base Network

In [67]:
input = Input(shape=(img_width, img_height, 3))
output_mobilenet_conv = model_mobilenet_conv(input)
x1 = Flatten()(output_mobilenet_conv)
x2 = Dense(64, activation='relu')(x1)
x3 = Dense(2, activation='softmax')(x2)
model = Model(input=input, output=x3)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Dataset Preparation

In [69]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=4, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                        batch_size=4,class_mode='categorical')

Found 1009 images belonging to 2 classes.
Found 17 images belonging to 2 classes.


# Training

In [70]:
callbacks = [ModelCheckpoint(model_weights_file, monitor='val_acc', save_best_only=True)]

history = model.fit_generator( train_generator, callbacks = callbacks, samples_per_epoch=nb_train_samples, 
                              nb_epoch=nb_epochs, validation_data=validation_generator, nb_val_samples=nb_val_samples)
model.save("mobile.h5")
print('Training Completed!')

Epoch 1/150
5/5 [==============================] - 11s 2s/step - loss: 26.5434 - accuracy: 0.4500 - val_loss: 16.7758 - val_accuracy: 0.5000
Epoch 2/150
5/5 [==============================] - 7s 1s/step - loss: 21.3408 - accuracy: 0.4500 - val_loss: 2.4008 - val_accuracy: 0.6154
Epoch 3/150
5/5 [==============================] - 6s 1s/step - loss: 3.6592 - accuracy: 0.8000 - val_loss: 3.4536 - val_accuracy: 0.4615
Epoch 4/150
5/5 [==============================] - 4s 870ms/step - loss: 0.8985 - accuracy: 0.9500 - val_loss: 12.5368 - val_accuracy: 0.3077
Epoch 5/150
5/5 [==============================] - 5s 993ms/step - loss: 2.6521 - accuracy: 0.8000 - val_loss: 26.2578 - val_accuracy: 0.4615
Epoch 6/150
5/5 [==============================] - 5s 1s/step - loss: 2.2657 - accuracy: 0.9000 - val_loss: 10.9218 - val_accuracy: 0.4375
Epoch 7/150
5/5 [==============================] - 6s 1s/step - loss: 0.9943 - accuracy: 0.7500 - val_loss: 23.0155 - val_accuracy: 0.6923
Epoch 8/150
5/5 [===

Epoch 59/150
5/5 [==============================] - 5s 1s/step - loss: 0.0092 - accuracy: 1.0000 - val_loss: 1.2811 - val_accuracy: 0.6154
Epoch 60/150
5/5 [==============================] - 5s 924ms/step - loss: 0.9038 - accuracy: 0.9000 - val_loss: 2.3842e-07 - val_accuracy: 0.3077
Epoch 61/150
5/5 [==============================] - 6s 1s/step - loss: 0.1577 - accuracy: 0.9500 - val_loss: 5.9782 - val_accuracy: 0.5625
Epoch 62/150
5/5 [==============================] - 6s 1s/step - loss: 0.0506 - accuracy: 0.9500 - val_loss: 1.8675 - val_accuracy: 0.5385
Epoch 63/150
5/5 [==============================] - 5s 923ms/step - loss: 2.1016 - accuracy: 0.7500 - val_loss: 3.3259 - val_accuracy: 0.2308
Epoch 64/150
5/5 [==============================] - 4s 847ms/step - loss: 0.5052 - accuracy: 0.9000 - val_loss: 0.9244 - val_accuracy: 0.6154
Epoch 65/150
5/5 [==============================] - 4s 885ms/step - loss: 2.6850 - accuracy: 0.7500 - val_loss: 6.1940 - val_accuracy: 0.5385
Epoch 66/15

In [71]:
# load and evaluate a saved model
from numpy import loadtxt
from keras.models import load_model

# load model
model_solid = load_model('mobile.h5')


# Testing

In [72]:
img_path = "mobile.png"
label = ['Degradable','non-degradable']
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model_solid.predict(x)
ind = np.argmax(features,axis=1)
print('Predicted Array:',features)
print('Predicted Label:',label[ind[0]])

Predicted Array: [[0.8074163  0.19258368]]
Predicted Label: Degradable
